In [22]:
import pandas as pd
import numpy as np
import glob
import re

## 鳳梨交易量排行榜
根據交易資料統計，台中交易市場為最大，因此選擇以台中為主要分析資料，農測站選擇高雄農改

## 交易資料 => 使用台中市場

In [13]:
df_price = pd.read_csv("cleaned data/價格_全國鳳梨交易資料(初步清洗).csv", usecols=[0, 1, 2, 3, 4, 5, 6, 7]) # 因稍候需要補差值，故略過百分比欄位
df_price.head()


,日期,市場,產品,上價,中價,下價,平均價(元/公斤),交易量(公斤)
0,2011-01-01,104 台北二,B2 鳳梨 金鑽鳳梨,29.4,23.0,13.2,22.3,3885
1,2011-01-02,104 台北二,B2 鳳梨 金鑽鳳梨,30.1,22.6,12.3,22.1,5468
2,2011-01-04,104 台北二,B2 鳳梨 金鑽鳳梨,28.3,21.6,10.1,20.7,5068
3,2011-01-05,104 台北二,B2 鳳梨 金鑽鳳梨,27.5,16.9,7.4,17.1,5212
4,2011-01-06,104 台北二,B2 鳳梨 金鑽鳳梨,31.4,19.7,9.0,19.9,620


In [14]:
## 只留下台中市場資料
df_price_taichung = df_price.loc[df_price["市場"] == "400 台中市"]

In [15]:
df_price_taichung["日期"] = pd.to_datetime(df_price_taichung["日期"])
df_price_taichung.set_index("日期", inplace=True)

<ipython-input-15-10e8266cc8ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_taichung["日期"] = pd.to_datetime(df_price_taichung["日期"])


In [16]:
df_price_taichung

,市場,產品,上價,中價,下價,平均價(元/公斤),交易量(公斤)
日期,,,,,,,
2011-01-01,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,17.3,12.0,17.0,17027
2011-01-02,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-04,400 台中市,B2 鳳梨 金鑽鳳梨,20.7,16.2,10.8,17.6,51764
2011-01-05,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,16.9,10.0,17.5,11247
2011-01-06,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,15.2,15.0,18.4,2354
...,...,...,...,...,...,...,...
2021-06-10,400 台中市,B2 鳳梨 金鑽鳳梨,29.0,18.2,9.7,21.0,163380
2021-06-11,400 台中市,B2 鳳梨 金鑽鳳梨,28.5,17.8,8.0,21.1,87697
2021-06-12,400 台中市,B2 鳳梨 金鑽鳳梨,28.0,15.2,5.7,20.3,82402


In [17]:
df_price_taichung.dtypes

市場            object
產品            object
上價           float64
中價           float64
下價           float64
平均價(元/公斤)    float64
交易量(公斤)        int64
dtype: object

### 補上週末六日(是否需要補，尚待討論＆結果觀察)
- 目前使用ffill（補前值），這方法不太好，雖補齊休假日空缺，但是否影響資料整體尚待評估 => 使用resample("D")，以日為單位重採集資料
- 重算百分比變化 => 使用**shift()**
- 交易量變動量大原因 => 星期一、逢年過節?

In [18]:
df_price_taichung = df_price_taichung.resample("D").ffill()
df_price_taichung

,市場,產品,上價,中價,下價,平均價(元/公斤),交易量(公斤)
日期,,,,,,,
2011-01-01,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,17.3,12.0,17.0,17027
2011-01-02,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-03,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-04,400 台中市,B2 鳳梨 金鑽鳳梨,20.7,16.2,10.8,17.6,51764
2011-01-05,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,16.9,10.0,17.5,11247
...,...,...,...,...,...,...,...
2021-06-10,400 台中市,B2 鳳梨 金鑽鳳梨,29.0,18.2,9.7,21.0,163380
2021-06-11,400 台中市,B2 鳳梨 金鑽鳳梨,28.5,17.8,8.0,21.1,87697
2021-06-12,400 台中市,B2 鳳梨 金鑽鳳梨,28.0,15.2,5.7,20.3,82402


In [8]:
# 計算價格＆交易量變動百分比 -> 如不需要稍後刪除
#df_price_taichung["價格變動百分比"] = round(df_price_taichung["平均價(元/公斤)"] / df_price_taichung["平均價(元/公斤)"].shift(1), 2) - 1
#df_price_taichung["交易量變動百分比"] = round(df_price_taichung["交易量(公斤)"] / df_price_taichung["交易量(公斤)"].shift(1), 2) - 1


In [9]:
#df_price_taichung[["價格變動百分比", "交易量變動百分比"]] = df_price_taichung[["價格變動百分比", "交易量變動百分比"]].fillna(0)

In [10]:
# 改成英文欄位以利後續使用
#eng_cols_name = ["Market", "Product", "High price", "Mid price", "Low price", "Average price", "Volume(kg)", "Price change rate", "Volume change rate"]
#df_price_taichung.columns = eng_cols_name

In [19]:
df_price_taichung

,市場,產品,上價,中價,下價,平均價(元/公斤),交易量(公斤)
日期,,,,,,,
2011-01-01,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,17.3,12.0,17.0,17027
2011-01-02,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-03,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-04,400 台中市,B2 鳳梨 金鑽鳳梨,20.7,16.2,10.8,17.6,51764
2011-01-05,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,16.9,10.0,17.5,11247
...,...,...,...,...,...,...,...
2021-06-10,400 台中市,B2 鳳梨 金鑽鳳梨,29.0,18.2,9.7,21.0,163380
2021-06-11,400 台中市,B2 鳳梨 金鑽鳳梨,28.5,17.8,8.0,21.1,87697
2021-06-12,400 台中市,B2 鳳梨 金鑽鳳梨,28.0,15.2,5.7,20.3,82402


### 新增欄位需求
- 移動平均（5日均價）

## 天氣資料 => 使用日月潭測站資料

### 主要欄位中文名稱

<img src='image/weather_columns(definition).png' width=1600>

### 異常值符號代表意義&處理方式

In [18]:
dict_symbol = {"Symbol":["T", "x", "&", "V", "/", "..."],
               "Explanation":["微量(小於 0.1mm)", "故障", "降水量資料累積於後", "風向不定", "不明", "無觀測"],
               "Replacement":[0.0, "ffill", "ffill", "ffill", "ffill", "ffill"]
              }
df_symbol = pd.DataFrame(dict_symbol)
df_symbol

,Symbol,Explanation,Replacement
0,T,微量(小於 0.1mm),0
1,x,故障,ffill
2,&,降水量資料累積於後,ffill
3,V,風向不定,ffill
4,/,不明,ffill
5,...,無觀測,ffill


### 中英欄位對照表

In [20]:
col_data = "觀測時間(LST)ObsTime、測站氣壓(hPa)StnPres、海平面氣壓(hPa)SeaPres、測站最高氣壓(hPa)StnPresMax、測站最高氣壓時間(LST)StnPresMaxTime、測站最低氣壓(hPa)StnPresMin、測站最低氣壓時間(LST)StnPresMinTime、氣溫(℃)Temperature、最高氣溫(℃)T Max、最高氣溫時間(LST)T Max Time、最低氣溫(℃)T Min、最低氣溫時間(LST)T Min Time、露點溫度(℃)Td dew point、相對溼度(%)RH、最小相對溼度(%)RHMin、最小相對溼度時間(LST)RHMinTime、風速(m/s)WS、風向(360degree)WD、最大陣風(m/s)WSGust、最大陣風風向(360degree)WDGust、最大陣風風速時間(LST)WGustTime、降水量(mm)Precp、降水時數(hr)PrecpHour、10分鐘最大降水量(mm)PrecpMax10、10分鐘最大降水起始時間(LST)PrecpMax10Time、一小時最大降水量(mm)PrecpHrMax、一小時最大降水量起始時間(LST)PrecpHrMaxTime、日照時數(hr)SunShine、日照率(%)SunShineRate、全天空日射量(MJ/㎡)GloblRad、能見度(km)VisbMean、A型蒸發量(mm)EvapA"
datas = [i for i in col_data.split("、")]
chi,eng =[],[]
for data in datas:
    chi.append(data.split(")")[0]+")")
    eng.append(data.split(")")[1])
df_columns_name = pd.DataFrame({"英文名稱":eng, 
                                "中文名稱":chi})
df_columns_name


,英文名稱,中文名稱
0,ObsTime,觀測時間(LST)
1,StnPres,測站氣壓(hPa)
2,SeaPres,海平面氣壓(hPa)
3,StnPresMax,測站最高氣壓(hPa)
4,StnPresMaxTime,測站最高氣壓時間(LST)
5,StnPresMin,測站最低氣壓(hPa)
6,StnPresMinTime,測站最低氣壓時間(LST)
7,Temperature,氣溫(℃)
8,T Max,最高氣溫(℃)
9,T Max Time,最高氣溫時間(LST)


In [135]:
# concat 2011~ 2021的資料，選擇欲使用之欄位
cols = ['日期', '平均地溫100cm(℃)',  '平均氣溫(℃)', '平均氣壓(hPa)', '平均相對溼度( %)','累積日射量(MJm2)','累積日照時數(hr)', '累計雨量(mm)']

df_weather = ""
for csv in glob.glob(r"cleaned data/Agricultural_Weather_Datas(merged)/高雄農改/*.csv"):
    if csv == r"cleaned data/Agricultural_Weather_Datas(merged)/高雄農改\2011_高雄農改.csv":
        df_weather = pd.read_csv(r"cleaned data/Agricultural_Weather_Datas(merged)/高雄農改\2011_高雄農改.csv", usecols=cols)
        continue
    df = pd.read_csv(csv, usecols=cols)
    df_weather = pd.concat([df_weather, df], sort=False)
    
df_weather

,日期,平均地溫100cm(℃),平均氣壓(hPa),平均氣溫(℃),平均相對溼度( %),累積日射量(MJm2),累積日照時數(hr),累計雨量(mm)
0,2011-01-01,24.1,1013.1,16.0,66.0,6.21,3.8,0.0
1,2011-01-02,24.0,1014.6,17.1,73.0,8.90,7.3,0.0
2,2011-01-03,23.9,1013.5,17.2,78.0,8.23,6.4,0.0
3,2011-01-04,23.8,1013.7,17.5,76.0,9.19,8.3,0.0
4,2011-01-05,23.7,1012.7,18.6,78.0,6.00,4.1,0.0
...,...,...,...,...,...,...,...,...
360,2021-12-27,0.0,0.0,0.0,0.0,0.00,0.0,0.0
361,2021-12-28,0.0,0.0,0.0,0.0,0.00,0.0,0.0
362,2021-12-29,0.0,0.0,0.0,0.0,0.00,0.0,0.0
363,2021-12-30,0.0,0.0,0.0,0.0,0.00,0.0,0.0


### 過濾掉今天以後的日期(尚未有資料產生) =>因為現有資料只到 6/14，之後資料更新後再做修改！

In [146]:
df_weather["日期"] = df_weather["日期"].apply(lambda x: pd.Timestamp(x))
df_weather = df_weather[df_weather["日期"] < pd.Timestamp("2021-06-15")]

df_weather["日期"] = pd.to_datetime(df_weather["日期"])

<ipython-input-146-e2853b0a8fdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weather["日期"] = df_weather["日期"].apply(lambda x: pd.Timestamp(x))


In [149]:
df_weather.head()

,日期,平均地溫100cm(℃),平均氣壓(hPa),平均氣溫(℃),平均相對溼度( %),累積日射量(MJm2),累積日照時數(hr),累計雨量(mm)
0,2011-01-01,24.1,1013.1,16.0,66.0,6.21,3.8,0.0
1,2011-01-02,24.0,1014.6,17.1,73.0,8.90,7.3,0.0
2,2011-01-03,23.9,1013.5,17.2,78.0,8.23,6.4,0.0
3,2011-01-04,23.8,1013.7,17.5,76.0,9.19,8.3,0.0
4,2011-01-05,23.7,1012.7,18.6,78.0,6.00,4.1,0.0


In [133]:
df_price_taichung.head()

,市場,產品,上價,中價,下價,平均價(元/公斤),交易量(公斤)
日期,,,,,,,
2011-01-01,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,17.3,12.0,17.0,17027
2011-01-02,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-03,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476
2011-01-04,400 台中市,B2 鳳梨 金鑽鳳梨,20.7,16.2,10.8,17.6,51764
2011-01-05,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,16.9,10.0,17.5,11247


### 合併dataframe

In [152]:
df_merged = df_price_taichung.merge(df_weather, how="left", on="日期").drop(columns=['市場', '上價', '中價', '下價'])
df_merged

,日期,產品,平均價(元/公斤),交易量(公斤),平均地溫100cm(℃),平均氣壓(hPa),平均氣溫(℃),平均相對溼度( %),累積日射量(MJm2),累積日照時數(hr),累計雨量(mm)
0,2011-01-01,B2 鳳梨 金鑽鳳梨,17.0,17027,24.1,1013.1,16.0,66.0,6.21,3.8,0.0
1,2011-01-02,B2 鳳梨 金鑽鳳梨,17.4,23476,24.0,1014.6,17.1,73.0,8.90,7.3,0.0
2,2011-01-03,B2 鳳梨 金鑽鳳梨,17.4,23476,23.9,1013.5,17.2,78.0,8.23,6.4,0.0
3,2011-01-04,B2 鳳梨 金鑽鳳梨,17.6,51764,23.8,1013.7,17.5,76.0,9.19,8.3,0.0
4,2011-01-05,B2 鳳梨 金鑽鳳梨,17.5,11247,23.7,1012.7,18.6,78.0,6.00,4.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3813,2021-06-10,B2 鳳梨 金鑽鳳梨,21.0,163380,28.0,1003.0,28.5,82.0,28.39,9.7,3.5
3814,2021-06-11,B2 鳳梨 金鑽鳳梨,21.1,87697,28.0,1003.9,28.4,85.0,27.08,9.2,14.5
3815,2021-06-12,B2 鳳梨 金鑽鳳梨,20.3,82402,28.0,1006.4,28.8,82.0,26.11,8.6,0.5
3816,2021-06-13,B2 鳳梨 金鑽鳳梨,19.4,85591,28.1,1006.1,29.5,77.0,31.76,11.4,3.0


### 新增欄位(五日均溫)

In [157]:
df_merged["5日平均地溫100cm(℃)"] = df_merged["平均地溫100cm(℃)"].shift(5)
df_merged["5日平均累積日照時數(hr)"] = df_merged["累積日照時數(hr)"].shift(5)
df_merged["3日累計雨量(mm)"] = df_merged["累計雨量(mm)"].shift(3)


In [161]:
df_test = df_merged.dropna(how="any")
df_test.head()

,日期,產品,平均價(元/公斤),交易量(公斤),平均地溫100cm(℃),平均氣壓(hPa),平均氣溫(℃),平均相對溼度( %),累積日射量(MJm2),累積日照時數(hr),累計雨量(mm),5日平均地溫100cm(℃),5日平均累積日照時數(hr),3日累計雨量(mm)
5,2011-01-06,B2 鳳梨 金鑽鳳梨,18.4,2354,23.7,1013.3,18.5,85.0,4.05,0.0,0.5,24.1,3.8,0.0
6,2011-01-07,B2 鳳梨 金鑽鳳梨,18.5,11185,23.6,1015.5,14.8,82.0,2.46,0.0,0.0,24.0,7.3,0.0
7,2011-01-08,B2 鳳梨 金鑽鳳梨,18.1,10245,23.5,1013.4,17.9,78.0,8.11,7.3,0.0,23.9,6.4,0.0
8,2011-01-09,B2 鳳梨 金鑽鳳梨,18.6,7986,23.5,1011.7,19.6,77.0,7.43,5.9,0.0,23.8,8.3,0.5
9,2011-01-10,B2 鳳梨 金鑽鳳梨,18.6,7986,23.4,1011.8,17.6,77.0,5.39,2.1,0.0,23.7,4.1,0.0


### 輸出為csv檔

In [162]:
df_test.to_csv("analysis data/鳳梨分析資料.csv", index=False)

In [49]:
### 重新排序欄位

cols = list(df_test.columns)
df_test = df_test[[cols[-1]] + cols[:-1]]

df_test["日期"] = pd.to_datetime(df_test["日期"])

df_test.set_index("日期", drop=True, inplace=True)

# 取代原本的df_weather_taichung
df_weather_taichung = df_test

df_weather_taichung

## 使用index以leftjoin方式，合併兩個dataframe

In [50]:
df_merged = df_price_taichung.merge(df_weather_taichung, how="left", on="日期")

In [51]:
df_merged[[ "Price change rate", "Volume change rate"]] = df_merged[[ "Price change rate", "Volume change rate"]].apply(lambda x: round(x, 2))


In [52]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3818 entries, 2011-01-01 to 2021-06-14
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Market              3818 non-null   object 
 1   Product             3818 non-null   object 
 2   High price          3818 non-null   float64
 3   Mid price           3818 non-null   float64
 4   Low price           3818 non-null   float64
 5   Average price       3818 non-null   float64
 6   Volume(kg)          3818 non-null   int64  
 7   Price change rate   3818 non-null   float64
 8   Volume change rate  3818 non-null   float64
 9   StnPres             3818 non-null   float64
 10  SeaPres             3818 non-null   float64
 11  StnPresMax          3818 non-null   float64
 12  StnPresMin          3818 non-null   float64
 13  Temperature         3818 non-null   float64
 14  T Max               3818 non-null   float64
 15  T Min               3818 non-null   f

In [53]:
df_merged

,Market,Product,High price,Mid price,Low price,Average price,Volume(kg),Price change rate,Volume change rate,StnPres,...,PrecpHour,PrecpMax10,PrecpMax60,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,Cloud Amount
日期,,,,,,,,,,,,,,,,,,,,,
2011-01-01,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,17.3,12.0,17.0,17027,0.00,0.00,903.2,...,0.0,0.0,0.0,9.6,90.2,12.96,10.0,1.8,5,1.3
2011-01-02,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476,0.02,0.38,905.0,...,0.0,0.0,0.0,9.7,91.1,13.20,10.8,2.0,6,1.7
2011-01-03,400 台中市,B2 鳳梨 金鑽鳳梨,21.2,18.1,11.7,17.4,23476,0.00,0.00,904.4,...,0.0,0.0,0.0,1.1,10.3,3.39,11.0,0.4,3,8.8
2011-01-04,400 台中市,B2 鳳梨 金鑽鳳梨,20.7,16.2,10.8,17.6,51764,0.01,1.20,904.7,...,0.0,0.0,0.0,6.4,60.0,11.96,14.0,-1.4,6,5.8
2011-01-05,400 台中市,B2 鳳梨 金鑽鳳梨,20.0,16.9,10.0,17.5,11247,-0.01,-0.78,904.5,...,4.4,0.5,0.5,5.1,47.8,10.17,17.0,-1.2,6,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-10,400 台中市,B2 鳳梨 金鑽鳳梨,29.0,18.2,9.7,21.0,163380,-0.01,0.24,898.6,...,6.2,2.5,6.0,4.4,32.5,16.21,22.5,-2.8,8,7.2
2021-06-11,400 台中市,B2 鳳梨 金鑽鳳梨,28.5,17.8,8.0,21.1,87697,0.00,-0.46,899.3,...,0.3,0.5,0.5,3.8,28.1,16.23,25.5,-2.0,9,7.5
2021-06-12,400 台中市,B2 鳳梨 金鑽鳳梨,28.0,15.2,5.7,20.3,82402,-0.04,-0.06,901.7,...,6.2,7.5,14.5,2.8,20.7,14.51,7.5,-3.1,9,7.8


In [54]:
# 輸出合併檔案
df_merged.to_csv("analysis data/台中_鳳梨合併資料(價格&天氣).csv")

In [21]:
df_weather_taichung = pd.read_csv("cleaned data/weather_data/日月潭.csv")
df_weather_taichung

,ObsTime,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,...,PrecpMax60,PrecpMax60Time,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount
0,1,904.3,1530.8,905.3,2010-01-01 09:24,903.3,2010-01-01 14:12,13.4,16.3,2010-01-01 14:29,...,0.0,...,0.1,0.9,4.63,20.0,0.9,2,2010-01-01 11:00,9.5
1,2,903.1,1522.7,904.8,2010-01-02 00:04,901.7,2010-01-02 14:32,14.6,17.6,2010-01-02 11:09,...,0.7,2010-01-02 20:35,1.1,10.3,5.22,20.9,-0.5,2,2010-01-02 12:00,9.2
2,3,902.5,1513.9,904.1,2010-01-03 08:05,900.8,2010-01-03 14:29,12.6,18.1,2010-01-03 13:21,...,6.0,2010-01-03 04:29,4.2,39.4,9.23,14.2,-0.9,5,2010-01-03 13:00,8.4
3,4,902.3,1512.9,903.5,2010-01-04 09:31,900.9,2010-01-04 14:15,13.8,19.7,2010-01-04 13:38,...,0.0,...,8.1,76.0,14.42,19.7,0.6,5,2010-01-04 13:00,3.8
4,5,902.8,1518.9,904.8,2010-01-05 09:14,901.3,2010-01-05 13:47,13.7,19.3,2010-01-05 13:27,...,0.0,...,1.5,14.1,7.78,18.2,-0.7,3,2010-01-05 13:00,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,26,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,27,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,28,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,29,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [22]:
df_weather_taichung.dtypes

ObsTime            int64
StnPres           object
SeaPres           object
StnPresMax        object
StnPresMaxTime    object
StnPresMin        object
StnPresMinTime    object
Temperature       object
T Max             object
T Max Time        object
T Min             object
T Min Time        object
Td dew point      object
RH                object
RHMin             object
RHMinTime         object
WS                object
WD                object
WSGust            object
WDGust            object
WGustTime         object
Precp             object
PrecpHour         object
PrecpMax10        object
PrecpMax10Time    object
PrecpMax60        object
PrecpMax60Time    object
SunShine          object
SunShineRate      object
GloblRad          object
VisbMean          object
EvapA             object
UVI Max           object
UVI Max Time      object
Cloud Amount      object
dtype: object

In [23]:
# 為後續資料清洗，從df_weather_taichungd拷貝一份當成df_test，避免改動到原檔案導致重新讀取
# 刪除ObsTime欄位
df_test = df_weather_taichung.copy()
df_test.drop(columns="ObsTime", inplace=True)

In [24]:
df_test

,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,T Min,...,PrecpMax60,PrecpMax60Time,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount
0,904.3,1530.8,905.3,2010-01-01 09:24,903.3,2010-01-01 14:12,13.4,16.3,2010-01-01 14:29,11.1,...,0.0,...,0.1,0.9,4.63,20.0,0.9,2,2010-01-01 11:00,9.5
1,903.1,1522.7,904.8,2010-01-02 00:04,901.7,2010-01-02 14:32,14.6,17.6,2010-01-02 11:09,12.6,...,0.7,2010-01-02 20:35,1.1,10.3,5.22,20.9,-0.5,2,2010-01-02 12:00,9.2
2,902.5,1513.9,904.1,2010-01-03 08:05,900.8,2010-01-03 14:29,12.6,18.1,2010-01-03 13:21,10.4,...,6.0,2010-01-03 04:29,4.2,39.4,9.23,14.2,-0.9,5,2010-01-03 13:00,8.4
3,902.3,1512.9,903.5,2010-01-04 09:31,900.9,2010-01-04 14:15,13.8,19.7,2010-01-04 13:38,9.8,...,0.0,...,8.1,76.0,14.42,19.7,0.6,5,2010-01-04 13:00,3.8
4,902.8,1518.9,904.8,2010-01-05 09:14,901.3,2010-01-05 13:47,13.7,19.3,2010-01-05 13:27,10.8,...,0.0,...,1.5,14.1,7.78,18.2,-0.7,3,2010-01-05 13:00,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4196,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4197,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [25]:
# 去除 \xa0 = space
df_test = df_test.applymap(lambda x: x.replace("\xa0",""))

In [26]:
# 將"..."取代為np.nan
df_test = df_test.replace("...", np.nan)

In [27]:
# dropna => 刪除欄位全部為null的row
df_test.dropna(how="all", inplace=True)
df_test.tail()

,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,T Min,...,PrecpMax60,PrecpMax60Time,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount
4178,898.6,1493.7,900.4,2021-06-10 00:03,896.5,2021-06-10 16:00,22.4,28.5,2021-06-10 13:55,19.1,...,6.0,2021-06-10 00:01,4.4,32.5,16.21,22.5,-2.8,8,2021-06-10 14:00,7.2
4179,899.3,1502.2,901.8,2021-06-11 21:31,897.9,2021-06-11 07:02,23.2,28.6,2021-06-11 12:54,19.6,...,0.5,2021-06-11 19:59,3.8,28.1,16.23,25.5,-2.0,9,2021-06-11 13:00,7.5
4180,901.7,1524.9,903.6,2021-06-12 22:07,900.5,2021-06-12 16:20,22.5,28.9,2021-06-12 12:34,19.4,...,14.5,2021-06-12 16:06,2.8,20.7,14.51,7.5,-3.1,9,2021-06-12 13:00,7.8
4181,901.4,1522.1,903.1,2021-06-13 00:01,898.8,2021-06-13 16:51,23.1,29.4,2021-06-13 12:57,20.1,...,11.5,2021-06-13 17:03,4.9,35.4,20.10,22.5,-3.9,9,2021-06-13 13:00,6.2
4182,898.8,1498.2,901.2,2021-06-14 00:05,897.3,2021-06-14 16:57,23.5,29.0,2021-06-14 13:10,20.4,...,0.0,NaN,6.3,46.5,22.43,25.8,4.2,10,2021-06-14 12:00,5.8


In [28]:
# 新增欄位「日期」，做為稍後合併兩個dataframe的鍵值
df_test["日期"] =  df_test["StnPresMaxTime"].apply(lambda x: x[:10])
df_test

,StnPres,SeaPres,StnPresMax,StnPresMaxTime,StnPresMin,StnPresMinTime,Temperature,T Max,T Max Time,T Min,...,PrecpMax60Time,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,UVI Max Time,Cloud Amount,日期
0,904.3,1530.8,905.3,2010-01-01 09:24,903.3,2010-01-01 14:12,13.4,16.3,2010-01-01 14:29,11.1,...,NaN,0.1,0.9,4.63,20.0,0.9,2,2010-01-01 11:00,9.5,2010-01-01
1,903.1,1522.7,904.8,2010-01-02 00:04,901.7,2010-01-02 14:32,14.6,17.6,2010-01-02 11:09,12.6,...,2010-01-02 20:35,1.1,10.3,5.22,20.9,-0.5,2,2010-01-02 12:00,9.2,2010-01-02
2,902.5,1513.9,904.1,2010-01-03 08:05,900.8,2010-01-03 14:29,12.6,18.1,2010-01-03 13:21,10.4,...,2010-01-03 04:29,4.2,39.4,9.23,14.2,-0.9,5,2010-01-03 13:00,8.4,2010-01-03
3,902.3,1512.9,903.5,2010-01-04 09:31,900.9,2010-01-04 14:15,13.8,19.7,2010-01-04 13:38,9.8,...,NaN,8.1,76.0,14.42,19.7,0.6,5,2010-01-04 13:00,3.8,2010-01-04
4,902.8,1518.9,904.8,2010-01-05 09:14,901.3,2010-01-05 13:47,13.7,19.3,2010-01-05 13:27,10.8,...,NaN,1.5,14.1,7.78,18.2,-0.7,3,2010-01-05 13:00,7.0,2010-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,898.6,1493.7,900.4,2021-06-10 00:03,896.5,2021-06-10 16:00,22.4,28.5,2021-06-10 13:55,19.1,...,2021-06-10 00:01,4.4,32.5,16.21,22.5,-2.8,8,2021-06-10 14:00,7.2,2021-06-10
4179,899.3,1502.2,901.8,2021-06-11 21:31,897.9,2021-06-11 07:02,23.2,28.6,2021-06-11 12:54,19.6,...,2021-06-11 19:59,3.8,28.1,16.23,25.5,-2.0,9,2021-06-11 13:00,7.5,2021-06-11
4180,901.7,1524.9,903.6,2021-06-12 22:07,900.5,2021-06-12 16:20,22.5,28.9,2021-06-12 12:34,19.4,...,2021-06-12 16:06,2.8,20.7,14.51,7.5,-3.1,9,2021-06-12 13:00,7.8,2021-06-12
4181,901.4,1522.1,903.1,2021-06-13 00:01,898.8,2021-06-13 16:51,23.1,29.4,2021-06-13 12:57,20.1,...,2021-06-13 17:03,4.9,35.4,20.10,22.5,-3.9,9,2021-06-13 13:00,6.2,2021-06-13


In [29]:
# 刪除幾個不必要的時間欄位
df_columns = list(df_test.columns)
time_cols = list(filter(lambda x : re.findall(".*Time", x), df_columns))

In [30]:
time_cols

['StnPresMaxTime',
 'StnPresMinTime',
 'T Max Time',
 'T Min Time',
 'RHMinTime',
 'WGustTime',
 'PrecpMax10Time',
 'PrecpMax60Time',
 'UVI Max Time']

In [31]:
df_test.drop(columns=time_cols, inplace=True)

### 找出無法轉換資料型態的欄位，並對其進行處理

In [32]:
# transform to numeric type => 準備fail_col來存放失敗欄位，稍後處理
fail_col = []
for col in list(df_test.columns):
    try:
        df_test[col] = df_test[col].apply(pd.to_numeric)
    except:
        print(f"{col}:轉換失敗")
        fail_col.append(col)

Precp:轉換失敗
PrecpMax10:轉換失敗
PrecpMax60:轉換失敗
GloblRad:轉換失敗
EvapA:轉換失敗
UVI Max:轉換失敗
Cloud Amount:轉換失敗
日期:轉換失敗


In [33]:
df_test["WD"].sum()

1000160

In [50]:
# WD (wind direction)
# 將缺值（Ｘ）取代成眾數
# df_test.loc[~(df_test["WD"].str.isnumeric()), "WD"] = df_test["WD"].mode()[0]

In [34]:
# WSGust(最大陣風)
# unique()找出後解決
# 問題 => 如何快速找到不為float的欄位（isnumeric無效）

df_test["WSGust"].unique()
df_test.loc[(df_test["WSGust"] == "X"), "WSGust"] = np.nan
df_test["WSGust"].fillna(method="ffill", inplace=True)

In [32]:
# WDGust(最大陣風方向)
#df_test.loc[~(df_test["WDGust"].str.isnumeric()), "WDGust"] = df_test["WDGust"].mode()[0]


In [35]:
# Precp(降水量)
# 無雨時 => T 
# bad_data = list(filter(lambda x :re.match("[^(\d\.\d)]", x), list(df_test["Precp"])))

df_test.loc[(df_test["Precp"] == "T")]
df_test.loc[df_test["PrecpMax10"] == "T", "PrecpMax10"] = 0.0
df_test.loc[(df_test["Precp"] == "T"), ["Precp", "PrecpMax10", "PrecpMax60"]] = 0.0

In [36]:
# EvapA (A型蒸發量)
df_test.loc[(df_test["EvapA"] == "/"), "EvapA"] = np.nan
df_test["EvapA"].fillna(method="ffill", inplace=True)

In [56]:
df_test.loc[df_test["UVI Max"] == "X"]

,StnPres,SeaPres,StnPresMax,StnPresMin,Temperature,T Max,T Min,Td dew point,RH,RHMin,...,PrecpMax10,PrecpMax60,SunShine,SunShineRate,GloblRad,VisbMean,EvapA,UVI Max,Cloud Amount,日期
591,903.1,1538.7,904.5,902.0,23.4,30.2,20.3,20.4,84,65,...,0.0,0.0,5.7,44.1,15.44,20.3,3.0,X,6.8,2011-08-15
592,902.2,1530.2,903.7,900.8,23.3,30.4,19.7,19.6,80,65,...,0.5,0.5,7.4,57.4,16.51,16.5,-3.0,X,5.7,2011-08-16
593,901.8,1525.8,902.9,900.6,23.4,30.4,19.6,19.0,77,60,...,0.0,0.0,9.5,73.8,21.04,24.2,4.0,X,2.8,2011-08-17
594,902.0,1528.8,903.3,900.7,23.7,30.7,20.1,19.5,78,55,...,6.5,10.5,9.4,73.1,21.55,18.3,-6.1,X,3.7,2011-08-18
595,902.4,1530.5,903.7,901.3,22.8,28.8,18.6,19.6,83,67,...,0.0,0.0,7.5,58.4,18.84,20.8,4.9,X,6.5,2011-08-19
596,902.2,1529.1,903.2,901.2,23.1,30.1,19.3,20.5,85,65,...,0.0,0.0,5.2,40.6,15.01,11.3,3.9,X,6.3,2011-08-20
597,901.3,1520.3,902.6,899.9,22.9,29.7,20.2,20.0,84,65,...,0.0,0.0,4.3,33.6,15.71,8.0,3.4,X,5.0,2011-08-21
598,899.9,1507.1,901.1,898.6,23.1,28.4,20.2,19.8,82,67,...,0.0,0.0,7.1,55.6,19.14,5.0,2.3,X,5.8,2011-08-22
599,898.7,1495.0,900.1,897.2,22.8,28.8,19.9,19.7,83,64,...,0,0,6.5,51.0,14.48,7.3,-2.7,X,5.2,2011-08-23
600,898.4,1491.0,899.4,896.9,21.7,26.8,19.0,19.7,89,74,...,15.5,52.0,2.9,22.8,8.24,6.0,-4.2,X,6.8,2011-08-24


In [37]:
# UVI Max (最大紫外線)
df_test.loc[df_test["UVI Max"] == "X", "UVI Max"] = np.nan
df_test["UVI Max"].fillna(method="ffill", inplace=True)

In [38]:
df_test.iloc[2,:]

StnPres              902.5
SeaPres             1513.9
StnPresMax           904.1
StnPresMin           900.8
Temperature           12.6
T Max                 18.1
T Min                 10.4
Td dew point          11.7
RH                      95
RHMin                   69
WS                     1.7
WD                     290
WSGust                10.2
WDGust                 320
Precp                 18.0
PrecpHour              9.8
PrecpMax10             2.5
PrecpMax60             6.0
SunShine               4.2
SunShineRate          39.4
GloblRad              9.23
VisbMean              14.2
EvapA                 -0.9
UVI Max                  5
Cloud Amount           8.4
日期              2010-01-03
Name: 2, dtype: object

In [39]:
# Clound Amount
df_test.loc[(df_test["Cloud Amount"] == "/"), "Cloud Amount"] = np.nan
df_test["Cloud Amount"].fillna(method="ffill", inplace=True)

In [40]:
# GloblRad 
df_test.loc[(df_test['GloblRad'] == "X"), 'GloblRad'] = np.nan
df_test["GloblRad"].fillna(method="ffill", inplace=True)

### 轉換數值

注意：
- 類別型欄位 => WD、WDGust、WSGust、UVI Max
- 時間序列欄位 => 日期
- 其餘皆數值欄位

In [41]:
df_test.dtypes

StnPres         float64
SeaPres         float64
StnPresMax      float64
StnPresMin      float64
Temperature     float64
T Max           float64
T Min           float64
Td dew point    float64
RH                int64
RHMin             int64
WS              float64
WD                int64
WSGust          float64
WDGust            int64
Precp            object
PrecpHour       float64
PrecpMax10       object
PrecpMax60       object
SunShine        float64
SunShineRate    float64
GloblRad         object
VisbMean        float64
EvapA            object
UVI Max          object
Cloud Amount     object
日期               object
dtype: object

In [42]:
fail_col

['Precp',
 'PrecpMax10',
 'PrecpMax60',
 'GloblRad',
 'EvapA',
 'UVI Max',
 'Cloud Amount',
 '日期']

In [43]:
cols = ["Precp", 'PrecpMax10', 'PrecpMax60','EvapA', "GloblRad","UVI Max", "Cloud Amount"]
df_test[cols] = df_test[cols].apply(pd.to_numeric)

In [44]:
df_test.dtypes

StnPres         float64
SeaPres         float64
StnPresMax      float64
StnPresMin      float64
Temperature     float64
T Max           float64
T Min           float64
Td dew point    float64
RH                int64
RHMin             int64
WS              float64
WD                int64
WSGust          float64
WDGust            int64
Precp           float64
PrecpHour       float64
PrecpMax10      float64
PrecpMax60      float64
SunShine        float64
SunShineRate    float64
GloblRad        float64
VisbMean        float64
EvapA           float64
UVI Max           int64
Cloud Amount    float64
日期               object
dtype: object